In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from flaml.autogen.agentchat2.llm_chat import (
    LLMChatMessage,
    LLMChatAgent,
    LLMChatMessageStream,
    LLMChatContext,
    llm_chat_action,
    llm_chat_trigger,
)
from flaml.autogen import oai

In [3]:
stream = LLMChatMessageStream()

In [4]:
participant1 = LLMChatAgent(
    name="participant1",
    message_stream=stream,
    partner_name="participant2",
    system_message={
        "role": "system",
        "content": "You are debating with user whether it is better to travel to Yellow Stone from Seattle by car or by flight. Your position is that car is better.",
    },
    llm_config={"temperature": 0.9, "model": "gpt-3.5-turbo"},
)
participant2 = LLMChatAgent(
    name="participant2",
    message_stream=stream,
    partner_name="participant1",
    system_message={
        "role": "system",
        "content": "You are debating with user whether it is better to travel to Yellow Stone from Seattle by car or by flight. Your position is that flight is better.",
    },
    llm_config={"temperature": 0.9, "model": "gpt-3.5-turbo"},
)

In [5]:
stream.add_subscriber(participant1)
stream.add_subscriber(participant2)

2023-08-03 15:59:17,004 DEBUG Adding subscriber <flaml.autogen.agentchat2.llm_chat.LLMChatAgent object at 0x7ff88d5db110> to address participant1
2023-08-03 15:59:17,005 DEBUG Adding subscriber <flaml.autogen.agentchat2.llm_chat.LLMChatAgent object at 0x7ff88d5db210> to address participant2


In [6]:
participant1.register(lambda msg, ctx: llm_chat_trigger(msg, ctx) and len(ctx.chat_history) < 10, llm_chat_action)
participant2.register(lambda msg, ctx: llm_chat_trigger(msg, ctx) and len(ctx.chat_history) < 10, llm_chat_action)

In [7]:
stream.send(
    LLMChatMessage(
        sender_name="participant2",
        receiver_name="participant1",
        message={"role": "user", "content": "It is better to travel to Yellow Stone by flight."},
    ),
)

2023-08-03 15:59:17,061 DEBUG Send message LLMChatMessage(sender_name=participant2, receiver_name=participant1, message={'role': 'user', 'content': 'It is better to travel to Yellow Stone by flight.'}) to address participant1


In [8]:
stream.run()

2023-08-03 15:59:17,095 DEBUG Send message LLMChatMessage(sender_name=participant1, receiver_name=participant2, message={'role': 'user', 'content': 'I understand that flying might seem like a convenient option, but let me explain why traveling to Yellowstone by car is actually a better choice.\n\nFirstly, traveling by car offers a unique opportunity to enjoy the scenic beauty along the way. The drive from Seattle to Yellowstone National Park takes you through picturesque landscapes, such as the Cascade Mountains, Shoshone National Forest, and Grand Teton National Park. You can stop at various viewpoints, take photos, and appreciate the natural beauty that surrounds you.\n\nSecondly, traveling by car allows for more flexibility and independence. You have the freedom to create your own itinerary, make unplanned stops, and explore lesser-known attractions or hidden gems. This flexibility is especially valuable when visiting national parks like Yellowstone, where there are numerous trails,